# AUTOMATED PLANNING

## 0. Learning Objectives
By the end of this notebook, you will understand:
- How to represent planning problems using PDDL (Planning Domain Definition Language)
- The difference between classical and real-world planning
- How to model actions with preconditions and effects
- Various planning algorithms and their applications

We'll start by looking at `PlanningProblem` and `Action` data types for defining problems and actions. 
Then, we will see how to use them by trying to plan a trip from *Sibiu* to *Bucharest* across the familiar map of Romania (from [search.ipynb](https://github.com/aimacode/aima-python/blob/master/search.ipynb)), 
followed by some common planning problems and methods of solving them.

Let's start by importing everything from the planning module.

In [ ]:
from planning import *
from notebook import psource

---
## 1. Planning Problems

**PDDL** stands for **Planning Domain Definition Language** - a standardized way to describe planning problems that is widely used in AI research and industry applications.

The `PlanningProblem` class is used to represent planning problems in this module. Every planning problem needs three essential components:
* **Initial state**: What conditions are true at the beginning
* **Goals**: What conditions we want to achieve
* **Actions**: What operations we can perform to change the state

Think of it like planning a journey: you know where you start (initial state), where you want to go (goals), and what transportation options you have (actions).

View the source to see how the Python code implements these concepts.

In [ ]:
psource(PlanningProblem)

**Key Attributes Explained:**

The `init` attribute contains logical expressions that form the initial knowledge base for the problem - these are the facts that are true at the start.

The `goals` attribute contains expressions that represent what we want to achieve - our target conditions.

The `actions` attribute contains a list of `Action` objects that represent all possible operations we can perform.

**Key Methods:**
- `goal_test()`: Checks if we have reached our goal state
- `act()`: Executes a given action and updates the current state

This design follows the **state-space search** paradigm where planning becomes a search through possible world states.

### 1.1 ACTION

To model a planning problem properly, we need to represent **Actions** - the building blocks of any plan. Each action in PDDL requires three essential components:

* **Preconditions**: What must be true before the action can be executed
* **Effects**: What changes occur when the action is executed  
* **Name/Expression**: A symbolic representation of the action

Think of a simple action like "pick up a book":
- *Precondition*: The book must be on the table and your hand must be free
- *Effect*: You now have the book in your hand, and the book is no longer on the table

The planning module models actions using the `Action` class. Let's examine its implementation:

In [ ]:
psource(Action)

**How the Action Class Works:**

This class represents an action using three components: expression, preconditions, and effects.

**Key Features:**
- `precond` list stores the preconditions of the action
- `effect` list stores the effects that occur when the action executes
- **Negation handling**: Negative conditions use `~` symbol (e.g., `~At(obj, loc)`) which internally becomes `NotAt(obj, loc)`

**Why this negation approach?** By creating separate clauses for negative literals, we avoid maintaining two separate knowledge bases (positive and negative facts). This simplification makes algorithms like `GraphPlan` much more efficient.

**Important Methods:**
- `convert()`: Parses input strings and converts them to logical expressions
- `check_precond()`: Verifies if preconditions are satisfied in the current state
- `act()`: Executes the action on the given knowledge base

**Example**: For action "move from A to B":
- Precondition: `At(robot, A)` 
- Effect: `At(robot, B) & ~At(robot, A)` (now at B, no longer at A)

### 1.2 Building Our First Planning Problem: Romania Travel

Now let's apply these concepts by defining a planning problem using the tools we've learned. In particular, let's see if we can plan a trip across a simplified version of this map.

**Problem Setup**: Plan a route from Sibiu to Bucharest using available transportation options.

Here's our simplified map definition:

In [ ]:
from utils import *
# this imports the required expr so we can create our knowledge base

knowledge_base = [
    expr("Connected(Bucharest,Pitesti)"),
    expr("Connected(Pitesti,Rimnicu)"),
    expr("Connected(Rimnicu,Sibiu)"),
    expr("Connected(Sibiu,Fagaras)"),
    expr("Connected(Fagaras,Bucharest)"),
    expr("Connected(Pitesti,Craiova)"),
    expr("Connected(Craiova,Rimnicu)")
    ]

Now we add logical rules to complete our knowledge about traveling around the map. These represent the typical **symmetry** and **transitivity** properties of connections:

- **Symmetry**: If you can go from A to B, you can go from B to A
- **Transitivity**: If A connects to B and B connects to C, then A connects to C

These logical rules ensure our knowledge base understands what it truly means for two locations to be "connected" in the way humans understand it.

We also add our starting location: *Sibiu*.

In [ ]:
knowledge_base.extend([
     expr("Connected(x,y) ==> Connected(y,x)"),
     expr("Connected(x,y) & Connected(y,z) ==> Connected(x,z)"),
     expr("At(Sibiu)")
    ])

We now have a complete knowledge base representing our Romania travel domain:

In [ ]:
knowledge_base

**Defining Available Actions**

Now we define the possible actions for our travel problem. We have two types of transportation:

1. **Driving**: Between any connected cities (following road connections)
2. **Flying**: Direct flights between major airports

Based on [real Romanian airports](https://en.wikipedia.org/wiki/List_of_airports_in_Romania), we can fly directly between Sibiu, Bucharest, and Craiova.

Let's define these flight actions first:

In [ ]:
#Sibiu to Bucharest
precond = 'At(Sibiu)'
effect = 'At(Bucharest) & ~At(Sibiu)'
fly_s_b = Action('Fly(Sibiu, Bucharest)', precond, effect)

#Bucharest to Sibiu
precond = 'At(Bucharest)'
effect = 'At(Sibiu) & ~At(Bucharest)'
fly_b_s = Action('Fly(Bucharest, Sibiu)', precond, effect)

#Sibiu to Craiova
precond = 'At(Sibiu)'
effect = 'At(Craiova) & ~At(Sibiu)'
fly_s_c = Action('Fly(Sibiu, Craiova)', precond, effect)

#Craiova to Sibiu
precond = 'At(Craiova)'
effect = 'At(Sibiu) & ~At(Craiova)'
fly_c_s = Action('Fly(Craiova, Sibiu)', precond, effect)

#Bucharest to Craiova
precond = 'At(Bucharest)'
effect = 'At(Craiova) & ~At(Bucharest)'
fly_b_c = Action('Fly(Bucharest, Craiova)', precond, effect)

#Craiova to Bucharest
precond = 'At(Craiova)'
effect = 'At(Bucharest) & ~At(Craiova)'
fly_c_b = Action('Fly(Craiova, Bucharest)', precond, effect)

Now we define the driving action. Notice how this uses **variables** (x, y) to represent a general driving action between any two connected locations:

In [ ]:
#Drive
precond = 'At(x)'
effect = 'At(y) & ~At(x)'
drive = Action('Drive(x, y)', precond, effect)

Our goal is simple - reach Bucharest:

In [ ]:
goals = 'At(Bucharest)'

Finally, we define a goal test function that checks if we've reached our destination:

In [ ]:
def goal_test(kb):
    return kb.ask(expr('At(Bucharest)'))

With all components ready, we can now create our complete planning problem:

In [ ]:
prob = PlanningProblem(knowledge_base, goals, [fly_s_b, fly_b_s, fly_s_c, fly_c_s, fly_b_c, fly_c_b, drive])

---
### 1.3 Example of Planning Problems

#### Air Cargo Problem: A Classic Planning Example

The Air Cargo problem is a classic example in AI planning that demonstrates fundamental concepts clearly. 

**Problem Description**: 
- We have cargo at two airports: SFO (San Francisco) and JFK (New York)
- Goal: Exchange the cargo - send each cargo to the other airport
- Resources: Two airplanes to transport the cargo
- Available actions: Load, Unload, and Fly

This problem illustrates **resource management** and **sequential planning** - core challenges in real-world logistics.

Let's examine how this problem is defined in the module:

In [ ]:
psource(air_cargo)

**State Predicates in Air Cargo Problem:**

**Location predicates:**
- **At(c, a)**: Cargo 'c' is currently located at airport 'a'
- **~At(c, a)**: Cargo 'c' is NOT at airport 'a'

**Container predicates:**
- **In(c, p)**: Cargo 'c' is loaded inside plane 'p'
- **~In(c, p)**: Cargo 'c' is NOT inside plane 'p'

**Type declarations** (these define what objects exist):
- **Cargo(c)**: Declare 'c' as a cargo object
- **Plane(p)**: Declare 'p' as a plane object
- **Airport(a)**: Declare 'a' as an airport object

**Initial Situation**: 
- Cargo C1 and plane P1 are at SFO airport
- Cargo C2 and plane P2 are at JFK airport

**Goal State**: 
- Cargo C1 should be at JFK airport  
- Cargo C2 should be at SFO airport

Let's create an instance of this problem:

In [ ]:
airCargo = air_cargo()

Before executing any actions, let's check if the problem has already reached its goal (it shouldn't have):

In [ ]:
print(airCargo.goal_test())

As expected, it returns `False` because we haven't achieved the goal yet. 

Now we need to define a sequence of actions to solve this problem. The key insight is that we need to **coordinate multiple resources** (planes and cargo) to achieve our goal.

**Available Actions:**
- **Load(c, p, a)**: Load cargo 'c' into plane 'p' at airport 'a'
- **Fly(p, from, to)**: Fly plane 'p' from airport 'from' to airport 'to'  
- **Unload(c, p, a)**: Unload cargo 'c' from plane 'p' at airport 'a'

**Planning Strategy**: Think about the steps needed:
1. Load cargo into appropriate planes
2. Fly planes to destination airports  
3. Unload cargo at destinations

Here's one valid solution:

In [ ]:
solution = [expr("Load(C1 , P1, SFO)"),
            expr("Fly(P1, SFO, JFK)"),
            expr("Unload(C1, P1, JFK)"),
            expr("Load(C2, P2, JFK)"),
            expr("Fly(P2, JFK, SFO)"),
            expr("Unload (C2, P2, SFO)")] 

for action in solution:
    airCargo.act(action)

Now that we've executed our planned sequence of actions, let's verify if we've successfully achieved our goal:

In [ ]:
print(airCargo.goal_test())

Excellent! We have successfully achieved our goal. This demonstrates how **sequential planning** works - by breaking down a complex problem into a series of coordinated actions, we can achieve sophisticated logistics coordination.

#### The Spare Tire Problem: Sequential Actions with Dependencies

This is a practical everyday problem that demonstrates **action dependencies** - some actions must happen before others.

**Scenario**: You have a flat tire on your car and need to replace it with the spare tire from your trunk.

In [ ]:
psource(spare_tire)

**State Predicates:**
- **At(obj, loc)**: Object 'obj' is currently at location 'loc'
- **~At(obj, loc)**: Object 'obj' is NOT at location 'loc'  
- **Tire(t)**: Declare 't' as a tire object

**Key Locations**: Axle, Trunk, Ground

Let's create an instance of the spare tire problem:

In [ ]:
spareTire = spare_tire()

Let's check the initial state - has the spare tire already been mounted?

In [ ]:
print(spareTire.goal_test())

As expected, the goal hasn't been achieved yet. Now we need to plan the sequence of actions to mount the spare tire.

**Available Actions:**
- **Remove(obj, loc)**: Remove tire 'obj' from location 'loc'
- **PutOn(tire, Axle)**: Mount tire 'tire' onto the car's axle
- **LeaveOvernight()**: Special action - if we leave tires overnight, they get stolen! (This shows how planning problems can model real-world constraints)

**Logical Dependencies**: 
- We can only put the spare tire on the axle if it's been removed from the trunk
- We can only put a tire on the axle if the axle is clear (flat tire removed)

Here's one solution sequence:

In [ ]:
solution = [expr("Remove(Flat, Axle)"),
            expr("Remove(Spare, Trunk)"),
            expr("PutOn(Spare, Axle)")]

for action in solution:
    spareTire.act(action)

In [ ]:
print(spareTire.goal_test())

Perfect! This solution works. But what about **different orderings**? Let's test if the order of the Remove actions matters.

Here's an alternative solution:

In [ ]:
spareTire = spare_tire()

solution = [expr('Remove(Spare, Trunk)'),
            expr('Remove(Flat, Axle)'),
            expr('PutOn(Spare, Axle)')]

for action in solution:
    spareTire.act(action)

In [ ]:
print(spareTire.goal_test())

This demonstrates an important planning concept: **partial ordering**. Both solutions work because the order of the `Remove` actions doesn't matter - they can happen in any sequence as long as both occur before the `PutOn` action. This flexibility is important for efficient planning algorithms.

Success! We have mounted the spare tire and can continue our journey.

#### Three Block Tower Problem: The Sussman Anomaly

This problem involves **spatial reasoning** and demonstrates a famous challenge in AI planning called the **Sussman Anomaly**.

**Problem Domain**: 
- Cube-shaped blocks sitting on a table
- Blocks can be stacked, but only one block per stack position
- A robot arm can pick up and move blocks
- **Constraint**: Can only pick up the top block of any stack

**The Sussman Anomaly**: This particular configuration was named after Prof. Gerry Sussman because it reveals a fundamental limitation of simple planning approaches - sometimes you need to temporarily "undo" progress toward one goal to achieve another goal.

Let's examine how the `three_block_tower` problem is defined:

In [ ]:
psource(three_block_tower)

**State Predicates for Blocks World:**

**Spatial relationships:**
- **On(b, x)**: Block 'b' is directly on top of 'x' (x can be table or another block)
- **~On(b, x)**: Block 'b' is NOT on 'x'

**Object types:**
- **Block(b)**: Declare 'b' as a block object

**Accessibility:**
- **Clear(x)**: Nothing is on top of 'x', so it can be moved or used as a destination
- **~Clear(x)**: Something is on top of 'x', so it cannot be moved

**Why "Clear" matters**: The robot arm can only grasp objects that have a clear top surface.

Let's create an instance of this problem:

In [ ]:
threeBlockTower = three_block_tower()

Let's check if the blocks are already in the target configuration:

In [ ]:
print(threeBlockTower.goal_test())

The goal hasn't been achieved yet. Now we need to plan how to stack the blocks in the correct order.

**Available Actions:**
- **MoveToTable(b, x)**: Move block 'b' from on top of 'x' to the table (requires 'b' to be clear)
- **Move(b, x, y)**: Move block 'b' from 'x' to on top of 'y' (requires both 'b' and 'y' to be clear)

**The Challenge**: This is where the Sussman Anomaly appears - sometimes we need to move blocks that seem to be "correctly placed" in order to access other blocks we need to move first.

Here's the solution sequence:

In [ ]:
solution = [expr("MoveToTable(C, A)"),
            expr("Move(B, Table, C)"),
            expr("Move(A, Table, B)")]

for action in solution:
    threeBlockTower.act(action)

After executing our planned sequence, let's verify that we've achieved the goal configuration:

In [ ]:
print(threeBlockTower.goal_test())

Excellent! We've successfully built the tower in the specified order: A on B on C. This solution demonstrates how planning algorithms must sometimes make moves that seem counterproductive in the short term to achieve long-term goals.

#### Alternative Blocks World Formulation

The `three_block_tower` problem can also be defined more simply using just two types of actions: `ToTable(x, y)` and `FromTable(x, y)`. 

This demonstrates how the **same problem** can be represented with different action vocabularies - a key insight in planning research. The underlying spatial reasoning remains the same, but the action primitives are different.

Let's examine this alternative definition:

In [ ]:
psource(simple_blocks_world)

**State Predicates for Simple Blocks World:**

**Block-to-block relationships:**
- **On(x, y)**: Block 'x' is directly on block 'y' (both must be blocks)
- **~On(x, y)**: Block 'x' is NOT on block 'y'

**Table relationships:**
- **OnTable(x)**: Block 'x' is sitting directly on the table
- **~OnTable(x)**: Block 'x' is NOT on the table (it's on another block)

**Accessibility:**
- **Clear(x)**: Block 'x' has nothing on top of it
- **~Clear(x)**: Block 'x' has something stacked on top

Let's create an instance of this simpler formulation:

In [ ]:
simpleBlocksWorld = simple_blocks_world()

Let's check if this version has reached its goal:

In [ ]:
simpleBlocksWorld.goal_test()

As expected, the goal hasn't been achieved. Now let's solve it using the simpler action vocabulary.

**Available Actions:**
- **ToTable(x, y)**: Move block 'x' from on top of block 'y' to the table (requires 'x' to be clear)
- **FromTable(x, y)**: Move block 'x' from the table to on top of block 'y' (requires both 'x' and 'y' to be clear)

Notice how these actions are more specific about table interactions compared to the previous formulation.

Here's the solution:

In [ ]:
solution = [expr('ToTable(A, B)'),
            expr('FromTable(B, A)'),
            expr('FromTable(C, B)')]

for action in solution:
    simpleBlocksWorld.act(action)

Let's verify that this alternative approach also achieves the goal:

In [ ]:
print(simpleBlocksWorld.goal_test())

Success! This demonstrates that the same planning problem can be solved using different action representations. Both approaches achieve the same goal but use different fundamental operations.

#### Shopping Problem: Multi-Location Planning

This problem demonstrates **navigation and resource acquisition** - a common pattern in robotics and game AI.

**Scenario**: You need to acquire three items: milk, banana, and drill. Each item is available only at specific stores, and you must travel between locations to collect everything.

**Real-world applications**: Delivery routing, robot task planning, inventory management.

Let's examine how this problem is defined:

In [ ]:
psource(shopping_problem)

**State Predicates for Shopping:**

**Location tracking:**
- **At(x)**: We are currently at location 'x' (Home, SM=Supermarket, or HW=Hardware store)
- **~At(x)**: We are NOT currently at location 'x'

**Store inventory:**
- **Sells(store, item)**: Store 'store' has item 'item' available for purchase

**Personal inventory:**
- **Have(item)**: We possess item 'item'

**Key Insight**: This problem combines **spatial navigation** (moving between locations) with **resource acquisition** (buying items). The challenge is to plan an efficient route that visits all necessary stores.

In [ ]:
shoppingProblem = shopping_problem()

Let's check if we already have all the items we need:

In [ ]:
print(shoppingProblem.goal_test())

**Available Actions:**

- **Buy(item, store)**: Purchase 'item' from 'store' (requires being at that store and the store selling that item)
- **Go(from, to)**: Travel from location 'from' to location 'to'

**Planning Strategy**: We need to figure out an efficient route that visits all necessary stores. Since the supermarket sells both milk and bananas, we can get both items there, then go to the hardware store for the drill.

Here's an efficient solution that minimizes travel:

In [ ]:
solution = [expr('Go(Home, SM)'),
            expr('Buy(Milk, SM)'),
            expr('Buy(Banana, SM)'),
            expr('Go(SM, HW)'),
            expr('Buy(Drill, HW)')]

for action in solution:
    shoppingProblem.act(action)

We've executed our shopping plan. Let's verify that we've acquired all the required items:

In [ ]:
shoppingProblem.goal_test()

Perfect! Our shopping trip was successful. This demonstrates how planning can optimize real-world tasks like errand running by finding efficient routes and sequences.

#### Socks and Shoes Problem: Precedence Constraints

This simple problem illustrates **precedence constraints** - certain actions must happen before others.

**Real-world principle**: You must put on socks before shoes! This models many real situations where order matters.

In [ ]:
psource(socks_and_shoes)

**State Predicates:**
- **LeftSockOn**: Left sock has been put on
- **RightSockOn**: Right sock has been put on  
- **LeftShoeOn**: Left shoe has been put on
- **RightShoeOn**: Right shoe has been put on

**The Constraint**: Each shoe can only be put on after its corresponding sock is already on.

**Learning Point**: This demonstrates how preconditions in actions enforce logical ordering constraints.

In [ ]:
socksShoes = socks_and_shoes()

Let's check if we're already dressed:

In [ ]:
socksShoes.goal_test()

We need to get dressed! Let's plan a sequence that respects the sock-before-shoe constraint.

**Note**: The order of left vs. right doesn't matter, but socks must come before their corresponding shoes.

In [ ]:
solution = [expr('RightSock'),
            expr('RightShoe'),
            expr('LeftSock'),
            expr('LeftShoe')]

In [ ]:
for action in solution:
    socksShoes.act(action)
    
socksShoes.goal_test()

Excellent! We're now properly dressed. This simple example shows how planning systems can handle **ordering constraints** that are common in real-world processes.

#### Cake Problem: Logical Paradoxes in Planning

This famous problem demonstrates how planning systems handle **apparent contradictions** and shows the importance of **careful action design**.

**The Challenge**: Achieve the seemingly impossible state of both having a cake AND having eaten a cake simultaneously.

In [ ]:
psource(have_cake_and_eat_cake_too)

**State Predicates** (simplified propositional logic):
- **Have(Cake)**: We currently possess a cake
- **~Have(Cake)**: We do NOT currently possess a cake
- **Eaten(Cake)**: We have eaten a cake (this fact persists)

**The Insight**: The key is understanding that "eaten" is a **historical fact** that doesn't disappear, while "have" is a **current possession** that can change.

In [ ]:
cakeProblem = have_cake_and_eat_cake_too()

Let's check our initial state - do we currently have both a cake and the experience of having eaten one?

In [ ]:
print(cakeProblem.goal_test())

**Available Actions:**
- **Bake(x)**: Create a new cake 'x' (precondition: must not already have one)
- **Eat(x)**: Consume cake 'x' (creates permanent "eaten" fact, removes "have" fact)

**The Solution Strategy**: The order matters! We must eat the cake first (creating the "eaten" fact), then bake a new one (giving us possession again).

Here's the correct solution sequence:

In [ ]:
solution = [expr("Eat(Cake)"),
            expr("Bake(Cake)")]

for action in solution:
    cakeProblem.act(action)

Now let's verify our seemingly impossible achievement:

In [ ]:
print(cakeProblem.goal_test())

Success! We have achieved the goal of having our cake and eating it too. This demonstrates how careful modeling of action preconditions and effects can resolve apparent logical paradoxes.

**What happens if we try the wrong order?** Let's see why sequence matters in this problem:

In [ ]:
cakeProblem = have_cake_and_eat_cake_too()

solution = [expr('Bake(Cake)'),
            expr('Eat(Cake)')]

for action in solution:
    cakeProblem.act(action)

This raises an exception as expected! The error demonstrates a crucial planning principle: **precondition violations**. 

According to our problem definition, the `Bake(Cake)` action has a precondition `~Have(Cake)` - we can only bake a cake if we don't already have one. Since we started with a cake, trying to bake first violates this constraint.

This shows how planning systems enforce logical consistency and prevent invalid action sequences.

---
## 2. Real-World Planning

The classical planning problems we've seen so far make simplifying assumptions that don't hold in real applications. **Real-world planning** must handle:

- **Time constraints**: Actions take different amounts of time
- **Resource limitations**: Limited materials, equipment, or personnel  
- **Hierarchical decomposition**: Breaking complex tasks into subtasks
- **Uncertainty**: Actions might fail or have unexpected effects

### 2.1 RealWorldPlanningProblem Class

The `RealWorldPlanningProblem` class extends basic planning with these real-world considerations:

**Additional Features:**
- **Jobs**: Complex tasks that can be broken down into smaller actions
- **Resources**: Consumable materials and reusable equipment with availability tracking  
- **Time management**: Actions have durations and can run concurrently
- **Hierarchical planning**: High-level actions that decompose into primitive operations

**Enhanced Methods:**
- `refinements()`: Breaks down high-level actions into executable primitives
- `hierarchical_search()` and `angelic_search()`: Advanced planning algorithms for complex problems

This bridges the gap between academic toy problems and industrial planning applications.

In [ ]:
psource(RealWorldPlanningProblem)

### 2.2 HLA (High-Level Actions)

**High-Level Actions** represent complex operations that can be **hierarchically decomposed** into simpler primitive actions. This is crucial for managing complexity in real-world planning.

**Example**: "Assemble a car" is an HLA that breaks down into "Install engine", "Attach wheels", "Add interior", etc. Each of these might further decompose into even simpler actions.

In [ ]:
psource(HLA)

**HLA Class Extensions:**

Beyond basic preconditions and effects, HLA objects track real-world constraints:

**Temporal aspects:**
- `duration`: How long the action takes to complete

**Resource management:**
- **Consumable resources**: Materials that get used up (e.g., screws, fuel, raw materials)
- **Reusable resources**: Equipment that can be used repeatedly but may be busy (e.g., tools, machines, workers)

**Execution tracking:**
- `completed`: Boolean flag indicating if the HLA has finished

**Key Methods for Real-World Planning:**
- `do_action()`: Executes the action only if all resources are available
- `has_consumable_resource()`: Checks if enough materials are available
- `has_usable_resource()`: Verifies that reusable equipment is free
- `inorder()`: Ensures prerequisite tasks have been completed first

This resource and time management is essential for realistic planning in manufacturing, robotics, and project management.

---
### 2.3. REAL-WORLD PLANNING PROBLEMS

#### Job Shop Problem: Resource Scheduling

This problem models a common industrial scenario: **assembly line coordination** with limited resources.

**Scenario**: Assemble two cars simultaneously using shared resources like tools and workers.

**Key Challenges**:
- **Resource contention**: Both cars need the same equipment (engine hoist, wheel station)
- **Consumable management**: Limited supplies (lug nuts) that get used up
- **Time optimization**: Different operations take different amounts of time

**Jobs structure**: Each car needs [`AddEngine`, `AddWheels`, `Inspect`] in sequence.

This demonstrates how real planning systems handle **scheduling** and **resource allocation** - critical in manufacturing and project management.

In [ ]:
psource(job_shop_problem)

**State Predicates for Job Shop:**

**Assembly state:**
- **Has(car, component)**: Car 'car' has component 'component' (Engine or Wheel) installed
- **~Has(car, component)**: Car 'car' does NOT have component 'component'

**Quality assurance:**
- **Inspected(car)**: Car 'car' has passed inspection
- **~Inspected(car)**: Car 'car' has NOT been inspected

**Initial Setup:**
- Cars: C1, C2 (both starting without engines, wheels, or inspection)
- Engines: E1, E2 (available for installation)  
- Wheels: W1, W2 (available for installation)

**Goal**: Both cars should have engines and wheels installed, and both should be inspected.

**Real-world insight**: This models the challenge of coordinating parallel workflows with shared, limited resources.

Let's create the job shop problem:

In [ ]:
jobShopProblem = job_shop_problem()

Let's check the initial state - are any cars already complete?

In [ ]:
print(jobShopProblem.goal_test())

As expected, no cars are complete yet. Now we need to plan the assembly sequence, keeping in mind our **resource constraints**.

**Available Actions with Resource Requirements:**

**Engine installation:**
- **AddEngine1**: Install engine on car C1 (30 minutes, requires engine hoist)
- **AddEngine2**: Install engine on car C2 (60 minutes, requires engine hoist)

**Wheel installation:**
- **AddWheels1**: Install wheels on car C1 (30 minutes, requires wheel station + 20 lug nuts)
- **AddWheels2**: Install wheels on car C2 (15 minutes, requires wheel station + 20 lug nuts)

**Quality control:**
- **Inspect1**: Inspect car C1 (10 minutes, requires 1 inspector)
- **Inspect2**: Inspect car C2 (10 minutes, requires 1 inspector)

**Resource Contention**: Notice that both cars compete for the same equipment (engine hoist, wheel station, inspector). The planning system must schedule these operations to avoid conflicts.

Here's one possible solution that handles resource scheduling:

In [ ]:
solution = [jobShopProblem.jobs[1][0],
            jobShopProblem.jobs[1][1],
            jobShopProblem.jobs[1][2],
            jobShopProblem.jobs[0][0],
            jobShopProblem.jobs[0][1],
            jobShopProblem.jobs[0][2]]

for action in solution:
    jobShopProblem.act(action)

In [ ]:
print(jobShopProblem.goal_test())

Excellent! This solution demonstrates **effective resource scheduling**. By completing all operations on car C2 first, we avoided resource conflicts and successfully assembled both cars. This is one of many possible valid solutions - real planning systems explore multiple scheduling options to find optimal ones.

#### Double Tennis Problem: Multi-Agent Coordination

This problem introduces **multi-agent planning** where multiple actors must coordinate their actions to achieve a shared goal.

**Scenario**: Two tennis players (A and B) must work together to return an approaching ball. 

**Key Concepts**:
- **Distributed control**: Multiple agents act independently but must coordinate
- **Shared goals**: All agents work toward the same objective  
- **Spatial reasoning**: Players must be in the right positions
- **Flexible assignment**: It doesn't matter which player makes the return, as long as someone does

**Court locations**: LeftBaseLine, RightBaseLine, LeftNet, RightNet

**Real-world applications**: Robotics teams, distributed systems, collaborative AI

This represents a fundamental challenge in AI: how multiple intelligent agents can work together effectively.

In [ ]:
psource(double_tennis_problem)

**State Predicates for Multi-Agent Tennis:**

**Ball state:**
- **Approaching(Ball, location)**: The ball is approaching the specified court location
- **Returned(Ball)**: The ball has been successfully hit back (goal achieved!)

**Player positions:**
- **At(actor, location)**: Player 'actor' is currently at court 'location'  
- **~At(actor, location)**: Player 'actor' is NOT at court 'location'

**Important Note**: The goal state contains a **variable 'a'** in the expression `At(a, LeftNet) | At(a, RightNet)`. This means "there exists some agent at either LeftNet or RightNet" - it doesn't matter which specific player, as long as someone is in position.

This **flexible goal specification** is common in multi-agent systems where multiple agents can fulfill the same role.

Let's create the tennis problem:

In [ ]:
doubleTennisProblem = double_tennis_problem()

Let's check if the players are already in winning position:

In [ ]:
print(doubleTennisProblem.goal_test())

The goal hasn't been achieved yet. We need to coordinate the players to return the approaching ball successfully.

**Available Actions for Multi-Agent Coordination:**

**Ball interaction:**
- **Hit(actor, ball, location)**: Player 'actor' hits the 'ball' at 'location' (only works if the player is at the location where the ball is approaching)

**Player movement:**
- **Go(actor, destination, origin)**: Move 'actor' from 'origin' to 'destination'

**Multi-Agent Planning Insight**: Notice how actions must specify which agent performs them. This is different from single-agent planning where the actor is implicit.

**Variable Goals**: The goal state uses a variable `a` in expressions like `At(a, LeftNet)`. This means "there exists some agent at LeftNet" - the planning system can assign any available agent to fulfill this role. This flexibility is crucial for efficient multi-agent coordination.

**Strategy**: We need to get at least one player to the right position to return the ball, then have them execute the hit.

Here's a coordination solution:

In [ ]:
solution = [expr('Go(A, RightBaseLine, LeftBaseLine)'),
            expr('Hit(A, Ball, RightBaseLine)'),
            expr('Go(A, LeftNet, RightBaseLine)')]

for action in solution:
    doubleTennisProblem.act(action)

In [ ]:
doubleTennisProblem.goal_test()

Perfect! The team successfully returned the ball through coordinated action. Player A moved to the right position, made the return, and then positioned themselves strategically for the next play.

This demonstrates how **multi-agent planning** can coordinate multiple actors to achieve shared objectives - a fundamental capability for robotics teams, distributed systems, and collaborative AI applications.

---
## 3. Automated Planning Algorithms

So far we've seen how to manually construct solutions by specifying each action step-by-step. However, the real power of AI planning comes from **automated planning algorithms** that can find solutions automatically.

The `planning.py` module provides several sophisticated planning algorithms:

1. **Forward State-Space Search** - Searches from initial state toward goals
2. **Backward State-Space Search** - Searches from goals toward initial state  
3. **GraphPlan** - Uses planning graphs with mutex analysis
4. **CSP Planning** - Converts planning to constraint satisfaction
5. **SAT Planning** - Converts planning to boolean satisfiability

Let's demonstrate these algorithms on our example problems!

### 3.1 Forward State-Space Search

Forward planning searches from the initial state toward the goals by systematically applying actions. This is the most intuitive approach - we start where we are and try different action sequences until we reach our goals.

In [ ]:
# Import the search module for planning algorithms
import search

# Let's demonstrate forward planning on the spare tire problem
spareTire = spare_tire()
print("=== Spare Tire Problem - Forward Planning ===")
print(f"Initial state: {spareTire.initial}")
print(f"Goals: {spareTire.goals}")
print(f"Available actions: {[str(action) for action in spareTire.actions]}")
print()

In [ ]:
# Create a ForwardPlan search problem
forward_problem = ForwardPlan(spareTire)

# Use breadth-first search to find the optimal solution
solution = search.breadth_first_graph_search(forward_problem)

if solution:
    print("Forward Planning SUCCESS!")
    print(f"Solution found with {len(solution.solution())} actions:")
    for i, action in enumerate(solution.solution(), 1):
        print(f"  {i}. {action}")
    print()
    
    # Verify the solution works - convert actions to proper expressions
    test_problem = spare_tire()
    print("Verifying solution:")
    for action in solution.solution():
        print(f"  Executing: {action}")
        # Convert action back to expression format for PlanningProblem.act()
        action_expr = expr(str(action))
        test_problem.act(action_expr)
    print(f"  Goal achieved: {test_problem.goal_test()}")
else:
    print("No solution found")

**Forward Planning on the Cake Problem**

Let's try forward planning on a slightly more complex problem - the cake problem where we need to both have and eat cake!

In [ ]:
# Test forward planning on the cake problem
cake_problem = have_cake_and_eat_cake_too()
print("=== Cake Problem - Forward Planning ===")
print(f"Initial: {cake_problem.initial}")
print(f"Goals: {cake_problem.goals}")

# Create forward search problem
cake_forward = ForwardPlan(cake_problem)
cake_solution = search.breadth_first_graph_search(cake_forward)

if cake_solution:
    print("Solution found!")
    print("Actions needed:")
    for i, action in enumerate(cake_solution.solution(), 1):
        print(f"  {i}. {action}")
else:
    print("No solution found")

### 3.2 Backward State-Space Search

Backward planning works in reverse - it starts from the goals and searches backward to find actions that could achieve those goals. This can be more efficient when there are fewer ways to achieve the goals than to progress from the initial state.

In [ ]:
# Demonstrate backward planning on the spare tire problem
print("=== Spare Tire Problem - Backward Planning ===")
backward_problem = BackwardPlan(spareTire)

# Use breadth-first search for backward planning
backward_solution = search.breadth_first_graph_search(backward_problem)

if backward_solution:
    print("Backward Planning SUCCESS!")
    actions = backward_solution.solution()
    print(f"Solution found with {len(actions)} actions:")
    for i, action in enumerate(actions, 1):
        print(f"  {i}. {action}")
        
    # Note: Backward planning solution may need to be reversed for execution
    print("\nNote: Backward planning found the actions needed.")
    print("The search explored goal conditions backwards to find relevant actions.")
else:
    print("No solution found")

### 3.3 GraphPlan Algorithm

GraphPlan builds a planning graph that alternates between proposition levels (states) and action levels. It's particularly effective because it identifies **mutex relationships** - situations where actions or propositions are mutually exclusive and cannot occur simultaneously.

The `planning.py` module provides ready-to-use GraphPlan implementations for our example problems.

In [ ]:
# GraphPlan for Spare Tire Problem
print("=== Spare Tire Problem - GraphPlan ===")
graphplan_solution = spare_tire_graphPlan()

if graphplan_solution and graphplan_solution[0]:
    print("GraphPlan SUCCESS!")
    # GraphPlan returns a tuple (success, plan_levels)
    linear_solution = linearize(graphplan_solution)
    print(f"Solution with {len(linear_solution)} actions:")
    for i, action in enumerate(linear_solution, 1):
        print(f"  {i}. {action}")
else:
    print("GraphPlan failed to find solution")

print()

In [ ]:
# GraphPlan for the Cake Problem  
print("=== Cake Problem - GraphPlan ===")
cake_graphplan_solution = have_cake_and_eat_cake_too_graphPlan()

if cake_graphplan_solution:
    print("GraphPlan SUCCESS!")
    # The cake problem returns the solution directly
    print(f"Solution with {len(cake_graphplan_solution)} actions:")
    for i, action in enumerate(cake_graphplan_solution, 1):
        print(f"  {i}. {action}")
else:
    print("GraphPlan failed")

print()

In [ ]:
# GraphPlan for the challenging Sussman Anomaly (Three Block Tower)
print("=== Three Block Tower (Sussman Anomaly) - GraphPlan ===")
tower_solution = three_block_tower_graphPlan()

if tower_solution and tower_solution[0]:
    print("GraphPlan solved the Sussman Anomaly!")
    linear_tower = linearize(tower_solution)
    print(f"Solution with {len(linear_tower)} actions:")
    for i, action in enumerate(linear_tower, 1):
        print(f"  {i}. {action}")
    
    print("\nThis demonstrates non-linear planning:")
    print("- Cannot directly achieve 'On(A,B)' because C is on A")  
    print("- Cannot directly achieve 'On(B,C)' because B needs to be clear")
    print("- Solution requires clearing A first, then building tower from bottom up")
else:
    print("GraphPlan failed")

### 3.4 Comparing Planning Approaches

Let's compare the solutions found by different algorithms on the same problem to understand their characteristics:

In [ ]:
import time

def compare_planners(problem_name, problem_instance):
    """Compare different planning approaches on the same problem"""
    print(f"=== Comparing Planners on {problem_name} ===")
    
    results = {}
    
    # Forward Planning
    start_time = time.time()
    try:
        forward_prob = ForwardPlan(problem_instance)
        forward_sol = search.breadth_first_graph_search(forward_prob)
        forward_time = time.time() - start_time
        if forward_sol:
            results['Forward'] = {
                'success': True,
                'actions': len(forward_sol.solution()),
                'time': forward_time,
                'solution': forward_sol.solution()
            }
        else:
            results['Forward'] = {'success': False, 'time': forward_time}
    except Exception as e:
        results['Forward'] = {'success': False, 'error': str(e)}
    
    # Backward Planning  
    start_time = time.time()
    try:
        backward_prob = BackwardPlan(problem_instance)
        backward_sol = search.breadth_first_graph_search(backward_prob)
        backward_time = time.time() - start_time
        if backward_sol:
            results['Backward'] = {
                'success': True,
                'actions': len(backward_sol.solution()),
                'time': backward_time,
                'solution': backward_sol.solution()
            }
        else:
            results['Backward'] = {'success': False, 'time': backward_time}
    except Exception as e:
        results['Backward'] = {'success': False, 'error': str(e)}
    
    # Print results
    print(f"{'Method':<10} {'Success':<8} {'Actions':<8} {'Time (s)':<10}")
    print("-" * 40)
    for method, result in results.items():
        if result['success']:
            print(f"{method:<10} {'✅':<8} {result['actions']:<8} {result['time']:<10.4f}")
        else:
            print(f"{method:<10} {'❌':<8} {'N/A':<8} {result.get('time', 'N/A'):<10}")
    
    return results

In [ ]:
# Compare on spare tire problem
spare_results = compare_planners("Spare Tire", spare_tire())

In [ ]:
# Compare on cake problem
print()
cake_results = compare_planners("Cake Problem", have_cake_and_eat_cake_too())

### 3.5 CSP-Based Planning

Planning can also be formulated as a Constraint Satisfaction Problem (CSP). If we know (or can estimate) the solution length, we can create variables for each time step and action, then add constraints to ensure the plan is valid.

In [ ]:
# Demonstrate CSP planning on the cake problem
print("=== Cake Problem - CSP Planning ===")

# Try different solution lengths until we find one that works
for max_length in range(1, 5):
    print(f"Trying solution length {max_length}...")
    try:
        csp_solution = CSPlan(have_cake_and_eat_cake_too(), max_length)
        if csp_solution:
            print(f"CSP Planning SUCCESS with {len(csp_solution)} actions!")
            for i, action in enumerate(csp_solution, 1):
                print(f"  {i}. {action}")
            break
        else:
            print(f"  No solution found with length {max_length}")
    except Exception as e:
        print(f"  Error with length {max_length}: {e}")
else:
    print("CSP Planning failed to find solution")

### 3.6 SAT-Based Planning  

SAT Planning converts the planning problem into a Boolean Satisfiability problem. Like CSP planning, it requires knowing or guessing the solution length, but it can leverage powerful SAT solvers.

In [ ]:
# Demonstrate SAT planning on the spare tire problem
print("=== Spare Tire Problem - SAT Planning ===")

for max_length in range(1, 6):
    print(f"Trying solution length {max_length}...")
    try:
        sat_solution = SATPlan(spare_tire(), max_length)
        if sat_solution:
            print(f"SAT Planning SUCCESS with {len(sat_solution)} actions!")
            for i, action in enumerate(sat_solution, 1):
                print(f"  {i}. {action}")
            break
        else:
            print(f"  No solution found with length {max_length}")
    except Exception as e:
        print(f"  Error with length {max_length}: {e}")
else:
    print("SAT Planning failed")

### 3.7 Planning Algorithm Performance Analysis

Let's sumarrise the characteristics of different planning approaches:

- Forward Search:
    - Intuitive: follows natural progression from start to goal
    - May explore many irrelevant states
    - Goal-directed heuristics needed for efficiency

- Backward Search:
    - Goal-focused: only considers goal-relevant actions
    - Can be more efficient for goal-directed problems
    - May consider unreachable states
    - Regression can be complex with negative effects

- GraphPlan:
    - Polynomial space complexity
    - Effective mutex detection reduces search space
    - Can prove unsolvability when graph levels off
    - Limited to propositional planning
    - Can struggle with large numbers of objects

- CSP Planning:
    - Can find optimal solutions (shortest plans)
    - Leverages efficient CSP algorithms
    - Good for problems with known solution bounds
    - Requires guessing solution length
    - Can generate large CSPs for long horizons

- SAT Planning:
    - Leverages powerful SAT solvers
    - Can handle complex logical constraints
    - Optimal solutions when solution length known
    - Requires guessing solution length
    - Encoding can become very large

---
## 4. Interactive Planning Exercise

Try modifying the problems below and see how different planners perform. You can experiment with:

1. **Different search strategies** (breadth-first, depth-first, A*)
2. **Modified problem definitions** (different goals, constraints)
3. **New simple problems** you create yourself

This hands-on experimentation will help you understand when each planning approach works best!

In [ ]:
# Example: Create a simple custom planning problem
def simple_robot_problem():
    """A robot needs to move to a target location and pick up an object"""
    return PlanningProblem(
        initial="At(Robot, A) & At(Object, B)",
        goals="At(Robot, B) & Holding(Object)",
        actions=[
            Action("Move(x, y)", 
                   precond="At(Robot, x)", 
                   effect="At(Robot, y) & ~At(Robot, x)"),
            Action("PickUp(obj, loc)", 
                   precond="At(Robot, loc) & At(obj, loc)", 
                   effect="Holding(obj) & ~At(obj, loc)")
        ]
    )

# Test the custom problem with forward planning
print("=== Custom Robot Problem - Forward Planning ===")
robot_problem = simple_robot_problem()
robot_forward = ForwardPlan(robot_problem)
robot_solution = search.breadth_first_graph_search(robot_forward)

if robot_solution:
    print("Solution found!")
    for i, action in enumerate(robot_solution.solution(), 1):
        print(f"  {i}. {action}")
else:
    print("No solution found")

print("\nTry creating your own planning problems!")
print("   - Modify goals or initial states")
print("   - Add new actions")
print("   - Test with different planning algorithms")